在之前我们利用储存在一个集合中的数据之间的相对位置关系，实现了对搜索算法效率的提高。例如，已知一个列表是一个有序表，那么我们就可以利用二分法实现数据搜索，将算法的时间复杂度控制在对数级别上。在这一节中，我们将尝试进一步建立一种新的数据结构，基于它的搜索算法的时间复杂度为$O(1)$。这个概念被称为Hash 查找。

为了实现这一数据结构，当我们尝试去寻找某一数据时，我们需要知道更多关于这一数据项可能在哪些位置出现的信息。如果所有的数据项都在恰当的位置上，那么我们就可以利用对应关系到那个位置上看看该数据项是否存在。然而，情况通常都不会这么简单。

哈希表是一种数据的集合，其中的每个数据都通过某种特定的方式进行存储以方面日后的查找。哈希表的每一个位置叫做槽，能够存放一个数据项，并以从0开始递增的整数命名。例如，第一个槽记为0，下一个记为1，再下一个记为2，并以此类推。在初始条件下，哈希表中是没有任何数据的，即每个槽都是空的。我们可以利用列表实现一个散列表，它的每一个元素都被初始化为None。图1. 展示了一个长度m=11的哈希表，换言之，这个哈希表中有m个槽，它们被依次命名为0到10。
![图1. 有11个空槽的散列表](pic/5.05.1.png)
<center>图1. 有11个空槽的散列表</center>

某个数据项与在哈希表中存储它的槽之间的映射叫做哈希函数。哈希函数可以将任意一个数据项存储到集合中并返回一个介于槽命名区间（即0与m-1）的整数。假设我们有一列整数54、26、93、17、77、31。我们的第一个哈希函数，有时被称为“求余”，简单地将要存储的数据项与哈希表的大小相除，返回余数作为这个数据项的哈希值(h(item)=item%11)。表1. 给出了上面例子中所有数据项的哈希值。值得注意的是，为了保证求得的哈希值落在哈希表的大小内，这个求余得到哈希值的方法将广泛地运用于所有种类的哈希函数中。
<center>表1. 利用求余方法的简单散列函数</center>

数据项|	哈希值
-|-
54|	10
26|	4
93|	5
17|	6
77|	0
31|	9

一旦计算了哈希值，我们可以将每个数据项插入到指定位置的哈希表中，如图2. 所示。11个插槽中的6个现在已被占用。一般地，我们把槽被占据的比例叫做负载因子$\lambda = \frac {项数}{表大小}$，在这个例子中，$\lambda = \frac {6}{11}$。
![图2. 填入了6个数据项的散列表](pic/5.05.2.png)
<center>图2. 填入了6个数据项的散列表</center>

现在当我们想要查找一个数据项时，我们只需要使用哈希函数去计算得到这个数据项对应槽的名字并在这个槽中检查该数据项是否存在即可。这个搜索过程的时间复杂度为$O(1)$，因为通常计算出哈希值和得到哈希表在该位置的索引需要一定的时间。如果所有数据项都在哈希函数所规定的位置上，我们就已经得到了一个时间复杂度为常数量级的搜索算法。

但是你可能已经看出了这个方法的问题所在，它仅能在每一个数据项在哈希表中占有不同的槽的情况下才能正常运作。例如，如果我们在哈希表中插入一个新的数据项44，它的哈希值为0(44 % 11 = 0)。但是由于77也拥有相同的哈希值0，问题也就随之产生。根据这种求余的哈希函数，两个甚至多个数据就需要存储在同一个槽中。这种情况被称为冲突。显然，冲突导致哈希方法出现了问题。我们将在后面的章节讨论这些问题。

### 5.05.1 hash 函数
对于一组给定的数据项，将每个项映射到唯一槽的哈希函数被称为完美哈希函数。如果我们知道项和集合将永远不会改变，那么可以构造一个完美的哈希函数。不幸的是，给定任意的项集合，没有系统的方法来构建完美的哈希函数。幸运的是，我们不需要哈希函数是完美的，仍然可以提高性能。

总是具有完美哈希函数的一种方式是增加哈希表的大小，直到所有可能的数据项变化范围都被哈希表所包含。这保证每个项将具有唯一的槽。虽然这对于小数目的项是实用的，但是当可能项的数目大时是不可行的。例如，如果项是九位数的社保号码，则此方法将需要大约十亿个槽。如果我们只想存储 25 名学生的数据，我们将浪费大量的内存。

我们的目标是创建一个哈希函数，最大限度地减少冲突数，易于计算，并均匀分布在哈希表中。有很多常用的方法来扩展简单余数法。我们将在这里介绍其中几个。

分组求和法将项划分为相等大小的块（最后一块可能不是相等大小）。然后将这些块加在一起以求出哈希值。例如，如果我们的项是电话号码 436-555-4601，我们将取出数字，并将它们分成2位数（43,65,55,46,01）。43 + 65 + 55 + 46 + 01，我们得到 210。我们假设哈希表有 11 个槽，那么我们需要除以 11 。在这种情况下，210％11 为 1，因此电话号码 436-555-4601 映射到槽 1 。一些分组求和法会在求和之前每隔一个反转。对于上述示例，我们得到 43 + 56 + 55 + 64 + 01 = 219，其给出 219％11 = 10 。

用于构造哈希函数的另一数值技术被称为 平方取中法。我们首先对该项平方，然后提取一部分数字结果。例如，如果项是 44，我们将首先计算 44^2 = 1,936 。通过提取中间两个数字 93 ，我们得到 5（93％11）。表2. 展示了余数法和中间平方法下的项。
<center>表1. 求余法和平方取中法的比较</center>

数据项| 求余法结果| 平方取中法结果
-|-|-
54|	10|	3
26|	4|	7
93|	5|	9
17|	6|	8
77|	0|	4
31|	9|	6

我们也可以为非数字的数据项，例如字符串创建散列表，’cat’ 可以看做一个连续的ASCII数值。

In [1]:
ord('c')

99

In [2]:
ord('a')

97

In [3]:
ord('t')

116

然后我们可以将三个数值加起来，接着用求余法得到一个哈希值(如图3.)。下面代码展示了一个哈希函数，用来产生一个字符串和一个表的大小，并返回一个0到表大小-1的哈希值。
![图3. 用ASCII数值散列一个字符串](pic/5.05.3.png)
<center>图3. 用ASCII数值散列一个字符串</center>

In [4]:
def hash(astring, tablesize):
    sum = 0
    for pos in range(len(astring)):
        sum = sum + ord(astring[pos])

    return sum%tablesize

有趣的是，当我们用哈希函数纪录哈希值的时候，颠倒的字母构成的单词会得到相同的哈希值。为了纠正这一情况，我们可以将字母的位置作为权重因子。图4. 展示了一种可能利用权重因子的方式。
![图4. 用权重因子对一个字符串求散列值](pic/5.05.4.png)
<center>图4. 用权重因子对一个字符串求散列值</center>

你们还可以想出其他的一些方法去计算数据的哈希值。但是最重要的事情就是哈希函数必须足够高效以防止它成为占据存储空间和搜索进程的主要部分。如果哈希函数过于复杂，导致花费大量的时间去计算槽的名称，可能还不如进行简单的顺序搜索或者二分法搜索，这就失去了哈希的意义。

### 5.05.2 冲突解决方法
我们现在回到冲突问题。当两个数据哈希映射到相同的槽，我们必须用一种系统化的方法将第二个数据放到哈希表里。这个过程叫做冲突解决。正如我们前面提到的，如果哈希函数是完美的，冲突就不会出现。但是，既然这一般来说是不可能的，冲突解决就成为实现哈希很重要的一部分。

一种解决冲突的方法就是搜索哈希表并寻找另一个空的槽来存放这个有冲突的数据。一种简单的方法就是从发生冲突的位置开始顺序向下开始寻找，直到我们遇到第一个空的槽。注意到我们可能需要回到第一个槽(循环)来实现覆盖整个哈希表。这种冲突解决方法叫做开放地址，它试图在哈希表中去寻找下一个空的槽。通过系统地向后搜索每一个槽，我们将这种实现开放地址的技术叫做线性探测。

图5. 展示了通过简单的求余哈希函数对一个扩展的数据集合进行处理后的结果( 54 、26 、93 、17 、77 、31 、44 、55 、20 )。上面的表格1. 展示了原始数据的哈希值。图2. 展示了原始哈希表中存储的数据项。当我们尝试把44放到槽0的时候，冲突出现了。通过线性探测，我们一个接一个槽地检查，直到我们找到一个空位置。在这里，我们找到了槽1。

同样的，55本应该放在槽0，但是这里必须被放到槽2，因为它是接下来的第一个空槽。最后一个数据项是20，本应该放到槽9。但是由于槽9是满的，于是我们开始线性探测，我们先后查看了槽10，0，1，和2，最终我们找到了空槽3。
![图5. 利用线性探测解决冲突](pic/5.05.5.png)
<center>图5. 利用线性探测解决冲突</center>

一旦我们利用开放地址和线性探测建立一个哈希表，最重要的问题就是我们要利用相同的方法去搜索数据。假设我们寻找数据93，先计算其散列值得到5，接着查看槽5，发现是93，所以我们返回真。那如果我们寻找20呢？现在哈希值是9，但是槽9现在存放的是31。这时我们不能简单地返回假，因为我们知道这可能存在冲突。现在我们需要进行线性搜索，从槽10开始，直到我们找到数据20或者找到一个空槽。

线性探测法的一个缺点是产生集中的趋势：数据会在表中聚集。这意味着如果对于同一哈希值产生了许多冲突时，周边的一系列槽都将会被线性探测填充。这将会对正在被填入的新数据产生影响，就像我们之前看到的，将数字20加入散列表中一样。当一簇值加入到哈希表中槽0时必须跳过原槽直到找到新的空槽。这种集中正如图6. 所示。
![图6. 一簇对应槽0的数据](pic/5.05.6.png)
<center>图6. 一簇对应槽0的数据</center>

一种解决集中问题的方法是扩展线性探测技术，我们不再按顺序一个一个地寻找新槽，而是跳过一些槽，这样能更加平均地分配出现冲突的数据，进而潜在地减少集中问题出现的次数。图7. 展示了同样的数据项如何通过“+3”线性探测的方法解决冲突的。“+3”表示一旦一个冲突出现，我们将每次跳过两个槽来寻找下一个新的空槽。
![图7. 用“+3”法解决冲突](pic/5.05.7.png)
<center>图7. 用“+3”法解决冲突</center>

这种冲突时为数据寻找新槽的过程被称作重新哈希。重新哈希函数的形式是newhashvalue=rehash(oldhashvalue)，对于简单的线性探测，它的具体形式是rehash(pos)=(pos+1)%sizeoftable。对于“+3”形式的再散列，它的具体形式是rehash(pos)=(pos+3)%sizeoftable。归纳起来的形式是rehash(pos)=(pos+skip)%sizeoftable。需要注意的一点是，选择跳过的槽的个数必须保证所有槽最终都能被遍历。否则，有些槽将会被闲置。为了保证这一点，我们通常建议将槽的数目设置成质数，这也是我们在例子里设置11个槽的原因。

另一种线性探测方法叫做二次探测法。我们不是每次在冲突中选择跳过固定个数的槽，而是使用一个重新哈希函数使每次跳过槽的数量会依次增加1，3，5，7，9，以此类推。这意味着如果原槽为第h个，那么重新哈希时访问的槽为第h+1，h+4，h+9，h+16个，以此类推。换言之，二次探测法使用一个连续的完全平方数数列作为它的跳跃值。图8显示了我们的例子在运用二次探测法时的填充结果。
![图8. 利用二次探测法解决冲突](pic/5.05.8.png)
<center>图8. 利用二次探测法解决冲突</center>

另一个解决冲突的替代方法是允许每一个槽都能填充一串而不是一个数据(称作链)。链能允许多个数据填在哈希表中的同一个位置上。当冲突发生时，数据还是填在本应该位于的槽中。随着一个槽中填入的数据的增多，搜索的难度也就随之增加。图12显示了数据在用数据链方法填入哈希表的结果。
![图9. 用数据链方法解决冲突](pic/5.05.9.png)
<center>图9. 用数据链方法解决冲突</center>
当我们要搜索一个项时，我们使用哈希函数来生成它应该在的槽。由于每个槽都有一个集合，我们使用一种搜索技术来查找该项是否存在。优点是，平均来说，每个槽中可能有更少的项，因此搜索可能更有效。我们将在本节结尾处查看散列的分析。

### 5.05.3 实现 map 抽象数据类型
字典是Python中最有用的数据类型之一。你可以在其中存储键-值对。该键用于查找关联的值。我们经常将这个想法称为映射（map）。map抽象数据类型定义如下。该结构是键值对的无序集合。map 中的键都是唯一的，因此键和值之间存在一对一的关系。操作如下。
- Map() 创建一个新的 map 。它返回一个空的 map 集合。
- put(key，val) 向 map 中添加一个新的键值对。如果键已经在 map 中，那么用新值替换旧值。
- get(key) 给定一个键，返回存储在 map 中的值或 None。
- del 使用 del map[key] 形式的语句从 map 中删除键值对。
- len() 返回存储在 map 中的键值对的数量。
- in  对于key in map语句，如果给定的键在map中返回 True，否则为False。

字典的一个巨大的好处在于给定键时，我们能够迅速的找到与其关联的数据值。为了提供这种快速查找能力，我们需要一个支持高效搜索的实现。我们可以使用一个顺序列表或者使用二分法搜索，但是更好地方法是使用哈希表的方式。正如之前所描述的，在一个哈希表中搜索数据的时间复杂度在O(1)级别。

在下面代码中我们运用两个列表来创造一个哈希表类(HashTable class)来实现映射的数据结构类型。其中一个称为slots(槽)，用来存储键，另一个平行列表称作data，用来存储数据值。当我们查找一个键时，对应的data列表中的位置保存着对应的数据值。依照之前的想法，我们把键表当作一个哈希表来处理。注意到哈希表的初始大小为11。尽管哈希表的槽数的选择是任意的，但是将其定为一个质数还是很有意义的，这样能够使解决冲突问题的算法尽可能发挥最大作用。

In [5]:
class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size
        self.data = [None] * self.size

哈希函数运用的是简单的求余方法。这里的冲突解决技术是运用“+1”的线性探测。其中put函数假设最终一定能找到一个能让新的密钥填入的槽，除非它已经在self.slots中存在。基于这样的假设，它能够计算出最初的哈希值，如果发现对应的槽不为空时，调用重新哈希(rehash)函数直到找到空槽位置。如果一个非空的槽已经含有该键，那么就将其数据值替换为当前数据值。

In [6]:
def put(self,key,data):
    hashvalue = self.hashfunction(key,len(self.slots))

    if self.slots[hashvalue] == None:
        self.slots[hashvalue] = key
        self.data[hashvalue] = data
    else:
        if self.slots[hashvalue] == key:
            self.data[hashvalue] = data  #replace
        else:
            nextslot = self.rehash(hashvalue,len(self.slots))
            while self.slots[nextslot] != None and \
                                  self.slots[nextslot] != key:
                nextslot = self.rehash(nextslot,len(self.slots))

            if self.slots[nextslot] == None:
                self.slots[nextslot]=key
                self.data[nextslot]=data
            else:
                self.data[nextslot] = data #replace

def hashfunction(self,key,size):
    return key%size

def rehash(self,oldhash,size):
    return (oldhash+1)%size

相似的，get函数也是首先计算最初的哈希值。如果结果不在对应的槽中，重新哈希(reshash)函数就会被用来确定一下个可能存储该键的位置。注意到第15确保了我们没有再次回到原槽，保证了搜索操作不会陷入死循环。如果这种情况发生了,那么我们已经遍历所有可
能的槽,这个键一定是不存在的。

哈希表类(HashTable class)的最后一些操作提供了额外的字典类功能。我们重载了运算符__getitem__和__setitem__允许使用“[]”对字典进行访问。这表示一旦一个哈希表被建立，我们所熟悉的索引操作符都将是可用的。

In [7]:
def get(self,key):
    startslot = self.hashfunction(key,len(self.slots))

    data = None
    stop = False
    found = False
    position = startslot
    while self.slots[position] != None and  \
                           not found and not stop:
        if self.slots[position] == key:
            found = True
            data = self.data[position]
        else:
            position=self.rehash(position,len(self.slots))
            if position == startslot:
                stop = True
    return data

def __getitem__(self,key):
    return self.get(key)

def __setitem__(self,key,data):
    self.put(key,data)

下面的部分展示了哈希表类(HashTable class)的操作。首先我们建立一个哈希表并保存一些包含整数键和字符串数据值的数据。

In [10]:
class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size
        self.data = [None] * self.size

    def put(self,key,data):
        hashvalue = self.hashfunction(key,len(self.slots))

        if self.slots[hashvalue] == None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data  #replace
            else:
                nextslot = self.rehash(hashvalue,len(self.slots))
                while self.slots[nextslot] != None and \
                                      self.slots[nextslot] != key:
                    nextslot = self.rehash(nextslot,len(self.slots))

                if self.slots[nextslot] == None:
                    self.slots[nextslot]=key
                    self.data[nextslot]=data
                else:
                    self.data[nextslot] = data #replace

    def hashfunction(self,key,size):
        return key%size

    def rehash(self,oldhash,size):
        return (oldhash+1)%size

    def get(self,key):
        startslot = self.hashfunction(key,len(self.slots))

        data = None
        stop = False
        found = False
        position = startslot
        while self.slots[position] != None and  \
                               not found and not stop:
            if self.slots[position] == key:
                found = True
                data = self.data[position]
            else:
                position=self.rehash(position,len(self.slots))
                if position == startslot:
                    stop = True
        return data

    def __getitem__(self,key):
        return self.get(key)

    def __setitem__(self,key,data):
        self.put(key,data)

In [11]:
H=HashTable()

In [12]:
H[54]="cat"

In [13]:
H[26]="dog"

In [14]:
H[93]="lion"

In [15]:
H[17]="tiger"

In [16]:
H[77]="bird"

In [17]:
H[31]="cow"

In [18]:
H[44]="goat"

In [19]:
H[55]="pig"

In [20]:
H[20]="chicken"

In [21]:
H.slots

[77, 44, 55, 20, 26, 93, 17, None, None, 31, 54]

In [22]:
H.data

['bird',
 'goat',
 'pig',
 'chicken',
 'dog',
 'lion',
 'tiger',
 None,
 None,
 'cow',
 'cat']

接下来，我们将访问和修改哈希表中的一些项。键20对应的数据值被替换了

In [23]:
H[20]

'chicken'

In [24]:
H[17]

'tiger'

In [25]:
H[20]='duck'

In [26]:
H[20]

'duck'

In [27]:
H.data

['bird',
 'goat',
 'pig',
 'duck',
 'dog',
 'lion',
 'tiger',
 None,
 None,
 'cow',
 'cat']

In [28]:
print(H[99])

None


### 5.05.4 hash法分析
我们之前说过，在最好的情况下，哈希将提供 $O(1)$，恒定时间搜索。然而，由于冲突，比较的数量通常不是那么简单。即使对哈希的完整分析超出了本文的范围，我们可以陈述一些近似搜索项所需的比较数量的已知结果。

我们需要分析哈希表的使用的最重要的信息是负载因子 λ。概念上，如果 λ 小，则碰撞的机会较低，这意味着数据项更可能在它们所属的槽中。如果 λ 大，意味着表正在填满，则存在越来越多的冲突。这意味着冲突解决更困难，需要更多的比较来找到一个空槽。使用链接，增加的碰撞意味着每个链上的项数量增加。

和以前一样，我们将有一个成功的搜索结果和不成功的。对于使用具有线性探测的开放寻址的成功搜索，平均比较数大约为$\frac{1}{2}\left(1+\frac{1}{1-\lambda}\right)$，不成功的搜索为$\frac{1}{2}\left(1+\left(\frac{1}{1-\lambda}\right)^2\right)$。  如果我们使用链接，则对于成功的情况，平均比较数目是 1+λ/2，如果搜索不成功，则简单地是 λ 比较次数。